In [ ]:
# default_exp data.token_classification

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# data.token_classification

> This module contains the bits required to use the fastai DataBlock API and/or mid-level data processing pipelines to organize your data for token classification tasks (e.g., NER or named entity recognition, etc...).

In [ ]:
#export
import ast
from functools import reduce

import torch
from transformers import *
from fastai2.text.all import *

from blurr.utils import *
from blurr.data.core import *

In [ ]:
#hide
import pdb

from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


## Token classification tokenization, batch transform, and DataBlock methods

Token classification tasks attempt to predict a class for each token.  The idea is similar to that in image segmentation models where the objective is to predict a class for each pixel.  Such models are common in building named entity recognition (NER) systems.

In [ ]:
# ensures these cols are represented as lists (rather than string)
df_converters = {'tokens': ast.literal_eval, 'labels': ast.literal_eval, 'nested-labels': ast.literal_eval}

path = Path('./')
germ_eval_df = pd.read_csv(path/'germeval2014_sample.csv', converters=df_converters); len(germ_eval_df)

1000

In [ ]:
germ_eval_df.head()

,id,source,tokens,labels,nested-labels,ds_type
0,0,n-tv.de vom 26.02.2005 [2005-02-26],"[Schartau, sagte, dem, "", Tagesspiegel, "", vom, Freitag, ,, Fischer, sei, "", in, einer, Weise, aufgetreten, ,, die, alles, andere, als, überzeugend, war, "", .]","[B-PER, O, O, O, B-ORG, O, O, O, O, B-PER, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",train
1,1,welt.de vom 29.10.2005 [2005-10-29],"[Firmengründer, Wolf, Peter, Bree, arbeitete, Anfang, der, siebziger, Jahre, als, Möbelvertreter, ,, als, er, einen, fliegenden, Händler, aus, dem, Libanon, traf, .]","[O, B-PER, I-PER, I-PER, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-LOC, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",train
2,2,http://www.stern.de/sport/fussball/krawalle-in-der-fussball-bundesliga-dfb-setzt-auf-falsche-konzepte-1553657.html#utm_source=standard&utm_medium=rss-feed&utm_campaign=sport [2010-03-25],"[Ob, sie, dabei, nach, dem, Runden, Tisch, am, 23., April, in, Berlin, durch, ein, pädagogisches, Konzept, unterstützt, wird, ,, ist, allerdings, zu, bezweifeln, .]","[O, O, O, O, O, O, O, O, O, O, O, B-LOC, O, O, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",train
3,3,stern.de vom 21.03.2006 [2006-03-21],"[Bayern, München, ist, wieder, alleiniger, Top-, Favorit, auf, den, Gewinn, der, deutschen, Fußball-Meisterschaft, .]","[B-ORG, I-ORG, O, O, O, O, O, O, O, O, O, B-LOCderiv, O, O]","[B-LOC, B-LOC, O, O, O, O, O, O, O, O, O, O, O, O]",train
4,4,http://www.fr-online.de/in_und_ausland/sport/aktuell/1618625_Frings-schaut-finster-in-die-Zukunft.html [2008-10-24],"[Dabei, hätte, der, tapfere, Schlussmann, allen, Grund, gehabt, ,, sich, viel, früher, aufzuregen, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]",train


In [ ]:
labels = sorted(list(set([lbls for sublist in germ_eval_df.labels.tolist() for lbls in sublist])))
print(labels)

['B-LOC', 'B-LOCderiv', 'B-LOCpart', 'B-ORG', 'B-ORGpart', 'B-OTH', 'B-OTHderiv', 'B-OTHpart', 'B-PER', 'B-PERderiv', 'B-PERpart', 'I-LOC', 'I-LOCderiv', 'I-ORG', 'I-ORGpart', 'I-OTH', 'I-PER', 'O']


In [ ]:
task = HF_TASKS_AUTO.TokenClassification

pretrained_model_name = "bert-base-multilingual-cased"
n_labels = len(labels)

hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(pretrained_model_name, 
                                                                               task=task,
                                                                               config_kwargs={'num_labels': n_labels})
hf_arch, type(hf_config), type(hf_tokenizer), type(hf_model)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

('bert',
 transformers.configuration_bert.BertConfig,
 transformers.tokenization_bert.BertTokenizer,
 transformers.modeling_bert.BertForTokenClassification)

Below, we define a new class and transform for token classification targets/predictions.

In [ ]:
#export
class HF_TokenTensorCategory(TensorBase): pass

In [ ]:
#export
class HF_TokenCategorize(Transform):
    "Reversible transform of a list of category string to `vocab` id"
    
    def __init__(self, vocab=None, ignore_token=None, ignore_token_id=None):  
        self.vocab = None if vocab is None else CategoryMap(vocab)
        self.ignore_token = '[xIGNx]' if ignore_token is None else ignore_token
        self.ignore_token_id = CrossEntropyLossFlat().ignore_index if ignore_token_id is None else ignore_token_id
        
        self.loss_func, self.order = CrossEntropyLossFlat(ignore_index=self.ignore_token_id), 1

    def setups(self, dsets):
        if self.vocab is None and dsets is not None: self.vocab = CategoryMap(dsets)
        self.c = len(self.vocab)

    def encodes(self, labels):
        ids = [[self.vocab.o2i[lbl]] + [self.ignore_token_id]*(n_subtoks-1) for lbl, n_subtoks in labels] 
        return HF_TokenTensorCategory(reduce(operator.concat, ids))
    
    def decodes(self, encoded_labels): 
        return Category([(self.vocab[lbl_id]) for lbl_id in encoded_labels if lbl_id != self.ignore_token_id ])

`HF_TokenCategorize` modifies the fastai `Categorize` transform in a couple of ways.  First, it allows your targets to consist of a `Category` ***per*** token, and second, it uses the idea of an `ignore_token` to mask subtokens that don't need a prediction.  For example, the target of special tokens (e.g., pad, cls, sep) are set to `ignore_token` as are subsequent sub-tokens of a given token should more than 1 sub-token make it up.

In [ ]:
#export
def HF_TokenCategoryBlock(vocab=None, ignore_token=None, ignore_token_id=None):
    "`TransformBlock` for single-label categorical targets"
    return TransformBlock(type_tfms=HF_TokenCategorize(vocab=vocab, 
                                                       ignore_token=ignore_token,
                                                       ignore_token_id=ignore_token_id))

In [ ]:
show_doc(HF_TokenCategoryBlock)

<h4 id="HF_TokenCategoryBlock" class="doc_header"><code>HF_TokenCategoryBlock</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>HF_TokenCategoryBlock</code>(**`vocab`**=*`None`*, **`ignore_token`**=*`None`*, **`ignore_token_id`**=*`None`*)

`TransformBlock` for single-label categorical targets

Again, we define a custom class, `HF_TokenClassInput`, for the @typedispatched methods to use so that we can override how token classification inputs/targets are assembled, as well as, how the data is shown via methods like `show_batch` and `show_results`.

In [ ]:
#export
class HF_TokenClassInput(list): pass

In [ ]:
#export
class HF_TokenClassBatchTransform(HF_BatchTransform):
    def __init__(self, hf_arch, hf_tokenizer, **kwargs):
        super().__init__(hf_arch, hf_tokenizer, HF_TokenClassInput, **kwargs)
        
    def encodes(self, samples):  
        samples = super().encodes(samples)
        if (len(samples[0]) == 1): return samples
        
        target_cls = type(samples[0][1])
        updated_samples = []
        
        # we assume that first target = the categories we want to predict for each token
        for s in samples:
            idx_first_input_id = s[0]['special_tokens_mask'].tolist().index(0)
            targ_ids = target_cls([ el*-100 if (el == 1) else s[1][idx-idx_first_input_id].item() 
                                   for idx, el in enumerate(s[0]['special_tokens_mask']) ])
            updated_samples.append((s[0], targ_ids))
        
        return updated_samples

`HF_TokenClassBatchTransform` is used to turn any targets we don't want to include in the loss calcuation (e.g. padding, cls, sep, etc...).

In [ ]:
hf_batch_tfm = HF_TokenClassBatchTransform(hf_arch, hf_tokenizer)

blocks = (
    HF_TextBlock(hf_arch, hf_tokenizer, is_pretokenized=True,
                 hf_batch_tfm=hf_batch_tfm,
                 tok_kwargs={ 'return_special_tokens_mask': True }), 
    HF_TokenCategoryBlock(vocab=labels)
)

def get_y(inp):
    return [ (label, len(hf_tokenizer.tokenize(str(entity)))) for entity, label in zip(inp.tokens, inp.labels) ]

dblock = DataBlock(blocks=blocks, 
                   get_x=ColReader('tokens'),
                   get_y=get_y,
                   splitter=RandomSplitter())

Note in the example above we had to define a `get_y` in order to return both the entity we want to predict a category for, as well as, how many subtokens are used by the `hf_tokenizer` to represent it.  This is necessary for the input/target alignment discussed above.

In [ ]:
# dblock.summary(test_df)

In [ ]:
dls = dblock.dataloaders(germ_eval_df, bs=4)

In [ ]:
b = dls.one_batch()

In [ ]:
len(b), b[0]['input_ids'].shape, b[1].shape

(2, torch.Size([4, 512]), torch.Size([4, 512]))

In [ ]:
#export
@typedispatch
def show_batch(x:HF_TokenClassInput, y, samples, dataloaders=None, ctxs=None, max_n=6, **kwargs):  
    hf_tokenizer = dataloaders.valid.hf_tokenizer
    
    res = L()
    for inp, trg, sample in zip(x[0], y, samples):
        inp_targs = [ (hf_tokenizer.ids_to_tokens[tok_id.item()], lbl_id.item()) 
                     for tok_id, lbl_id in zip(inp, trg) 
                     if (tok_id.item() not in hf_tokenizer.all_special_ids) and lbl_id != -100 ]
        
        res.append([f'{[ (inp_trg[0], lbl) for inp_trg, lbl in zip(inp_targs, ast.literal_eval(sample[1])) ]}'])
        
    display_df(pd.DataFrame(res, columns=['token / target label'])[:max_n])
    return ctxs

In [ ]:
dls.show_batch(dataloaders=dls, max_n=2)

,token / target label
0,"[('Als', 'O'), ('Aus', 'O'), ('bietet', 'O'), ('sich', 'O'), ('hier', 'O'), ('das', 'O'), ('Laufe', 'O'), ('am', 'O'), ('Rad', 'O'), ('oder', 'O'), ('die', 'O'), ('aktive', 'O'), ('Mitglied', 'O'), ('bei', 'O'), ('einem', 'O'), ('Ren', 'O'), ('oder', 'O'), ('Cours', 'O'), ('an', 'O'), ('.', 'O')]"
1,"[('Bereits', 'O'), ('für', 'O'), ('die', 'O'), ('""', 'O'), ('Start', 'O'), ('""', 'O'), ('-', 'O'), ('stellten', 'O'), ('die', 'O'), ('Deutschen', 'B-LOCderiv'), ('ihren', 'O'), ('""', 'O'), ('Santana', 'B-OTH'), ('""', 'O'), ('als', 'O'), ('Entwicklung', 'O'), ('zur', 'O'), ('Verfügung', 'O'), ('.', 'O')]"


## Cleanup

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01a_data-language-modeling.ipynb.
Converted 01c_data-question-answering.ipynb.
Converted 01d_data-token-classification.ipynb.
Converted 01e_data-text-generation.ipynb.
Converted 02_modeling-core.ipynb.
Converted 02a_modeling-language-modeling.ipynb.
Converted 02c_modeling-question-answering.ipynb.
Converted 02d_modeling-token-classification.ipynb.
Converted 02e_modeling-text-generation.ipynb.
Converted index.ipynb.
